In [1]:
import pandas as pd 
import re as re 
import numpy as np
import time 
import multiprocessing
pd.set_option('display.max_colwidth', -1)
pd.set_option('chained_assignment',None)

geneColID = ["chromosome","source","type","start","end","score","strand","phase","gene_symbol","gene_ensID","length","entrezid"]
geneAnnotationDF = pd.read_csv('entrez_id/geneAnnotationsDF_Selected_entrezID.csv', sep=',', comment='#', low_memory=False, header=0, names=geneColID)
chromosomeColID = ['chromosome','source','type','start','end','score','strand','phase']
chromosomesDF = pd.read_csv('chromosomesDF.csv', sep=',', comment='#', low_memory=False, header=0, names=chromosomeColID)

In [2]:
#Dropping unneccesary stuff and resetting index from 0
chromosomesDF = chromosomesDF.drop(columns=['score', 'strand', 'phase'])
geneAnnotationDF = geneAnnotationDF.drop(columns=['score', 'phase'])
chromosomesDF = chromosomesDF.reset_index()
geneAnnotationDF = geneAnnotationDF.reset_index()

In [3]:
# Making a list of DataFrames to be used in future functions in var geneDFL
u = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
geneDFL = []
for c in u:
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == c]
    geneDF = geneDF.drop(columns=['index'])
    geneDFL.append(geneDF)

In [4]:
#I need random chromosome combined with random position (Somewhere between start and end). Returns list ordered by chr num with sites. 
# probability that a randomly chosen gene is in a certain chromosome. Ordered from 1 - MT,X,Y | As shown chromosome 1 has highest prob of selection
probabilityL = [0.10328990533320836,
                0.06298622176398913,
                0.056378292248570626,
                0.03875714687412129,
                0.043959133939450744,
                0.05164495266660418,
                0.04658355984628362,
                0.03463304902052676,
                0.03828849939075827,
                0.03786671665573156,
                0.06411097572406037,
                0.05136376417658637,
                0.01762114537444934,
                0.030180897928578122,
                0.030837004405286344,
                0.04199081450932608,
                0.05844034117536789,
                0.01405942450089043,
                0.0687974505576905,
                0.02746274252507264,
                0.01241915830911988,
                0.02230762020807948,
                0.0007029712250445215,
                0.04213140875433499,
                0.0031868028868684973]

def nBindingsites(n):   
    chrIDS = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
    randChr = chromosomesDF.sample(n, replace=True, weights=probabilityL) # Replace allows same row to be sampled again 
    randChr['BindingSite'] = (np.random.uniform(randChr.start, randChr.end).round()).astype(int) 
    randChrL = [] 
    for i in chrIDS: 
        DF = randChr.loc[randChr['chromosome'] == i]
        sortDF = DF.iloc[(DF['BindingSite'] - 0).argsort()] 
        randChrL.append(sortDF)
    return randChrL
    #Returns a DataFrame with Random sample number with random BindingSite

In [5]:
#MIGHT NOT NEED ANY OF THIS ACTUALLY COULD BE FASTER 
# Distance from Body
def distanceBody(start, end, site):
    return min(abs(start - site), abs(end - site))

# Distance from TSS
def distanceTSS(start, site):
    return abs(start - site)

#Returns True or False to wheather the binding site is within the window size to Body | Start Site below
def isNearBody(start, end, site, window):
    if abs(start - site) <= window:
        return True
    elif abs(end - site) <= window:
        return True
    else:
        return False 
    
def isNearTSS(start, site, window):
    if abs(start - site) <= window:
        return True
    else: 
        return False
#-------------------------------------------- ## Should there be anymore iterations? Checker function 
def inRange(start, site, window):
    if start > (site + window):
        return False 
    else:
        return True

# For: k = Infinity 

In [6]:
#Given specific chromosome, rdm binding site, window: Will calculate number of nearby gene(bodies) from its site.
def genesInWindowBody(chrNum, site, window):
    count = 0
    geneL = []
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearBody(row['start'], row['end'], site, window):
            count = count + 1
            geneL.append(row['gene_symbol'])
    return(count, geneL)

#Given specific chromosome, rdm binding site, window: Will calculate number of nearby gene(TSS) from the site.
def genesInWindowTSS(chrNum, site, window):
    count = 0
    geneL = []
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearTSS(row['start'], site, window):
            count = count + 1
            geneL.append(row['gene_symbol'])    
    return(count, geneL)

## <font color=#4286f4>Given a window: Find all the genes inside window for the dataframe of chromosomes</font>

In [7]:
# Takes DF and Window size: Returnes Total Gene count and List of Genes Near Body of GENE (K=INFINITY)
def geneCountNsitesBody(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        c = genesInWindowBody(row['chromosome'], row['BindingSite'], window)
        count = count + c[0]
        geneL.append(c[1])
    return(count, geneL)

# Takes DF and Window size: Returnes Total Gene count and List of Genes Near TSS of GENE (K=INFINITY)
def geneCountNsitesTSS(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        c = genesInWindowTSS(row['chromosome'], row['BindingSite'], window)
        count = count + c[0]
        geneL.append(c[1])
    return(count, geneL)

# For: k = 1

In [8]:
def closestGeneBody2(site, count, window):
    geneDF = geneDFL[count]
    startG = geneDF.iloc[(geneDF['start'] - site).abs().argsort()].head(1)
    endG = geneDF.iloc[(geneDF['end'] - site).abs().argsort()].head(1)
    bodyG = geneDF[(geneDF['start'] <= site) & (geneDF['end'] >= site)]
    
    if bodyG.empty == False: 
        return(bodyG['entrezid'].iloc[0])
    
    startDist = distanceBody(startG['start'].iloc[0], startG['end'].iloc[0], site)
    endDist = distanceBody(endG['start'].iloc[0], endG['end'].iloc[0], site)
    if startDist < endDist:
        if isNearBody(startG['start'].iloc[0], startG['end'].iloc[0], site, window):
            return(startG['entrezid'].iloc[0])
        else:
            return('No Result')
    else:
        if isNearBody(endG['start'].iloc[0], endG['end'].iloc[0], site, window):
            return(endG['entrezid'].iloc[0])
        else:
            return('No Result')
        

#Given specific chromosome, rdm binding site, window: Will calculate closest gene(TSS) from its site. and Distance
def closestGeneTSS(chrNum, site, window):
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == str(chrNum)]
    startG = geneDF.iloc[(geneDF['start'] - site).abs().argsort()].head(1)
    if startG.empty: 
        return('No Result')
    elif isNearTSS(startG['start'].iloc[0], site, window):
        return(startG['entrezid'].iloc[0])
    else:
        return('No Result')

In [9]:
def closestGenesNsitesBody2(DFL_BS, window):
    geneList = []
    for v in range(len(DFL_BS)):
        d = DFL_BS[v]
        if d.empty == False:
            for row in zip(d['BindingSite'].values):
                x = closestGeneBody2(row[0], v, window)
                geneList.append(x)         
    return geneList
     
# Takes DF and Window size: Returnes Total Gene count and List of CLOSEST Gene Near its TSS for every site (K = 1)
def closestGenesNsitesTSS(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        x = closestGeneTSS(row['chromosome'], row['BindingSite'], window)
        geneL.append(x[0])
        count = count + 1
    return(count, geneL)

In [10]:
# Trying Multiprocessing 
def closestGenesNsitesBodyFASTER(df, window):
    num_processes = multiprocessing.cpu_count()
    chunk_size = int(df.shape[0]/num_processes)
#     chunks = [df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]
    chunks = [df.iloc[i:i + chunk_size,:] for i in range(0, df.shape[0], chunk_size)]
    return chunks
#     geneL = []   
#     for row in zip(df['chromosome'].astype(str), df['BindingSite'].values):
#         x = closestGeneBody(row[0], row[1], window)
#         if x == 'No Result':
#             pass
#         else:
#             geneL.append(x)
#     return(geneL)

In [11]:
# Adds midpoint in gene and upper and lower bound values to geneDFL based on size of window or nearby genes 
def addWindow(window):
    geneWindowDFL = geneDFL
    outList = []
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        dfG = dfG.reset_index() # Reseting index from 0 to length so that it can be used for rest of the function 
        dfG = dfG.drop(columns=['index']) # Dropping old index

        dfIndexMax = len(dfG) - 1         
        for i, row in dfG.iterrows():
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + row['start']) # Middle of gene body 
            
            if i == 0 or i == dfIndexMax: # Case for start and end sites
                if i == 0:
                    dfG.at[i, 'lowB'] = max(row['start'] - window, 1) # end of the gene below position
                    dfG.at[i, 'upperB'] = min(row['end'] + window, ((dfG.iat[i+1, 3] - row['end']) / 2) + row['end'])
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(row['start'] - window, row['start'] - ((row['start'] - dfG.iat[i-1, 4]) / 2)) 
                    dfG.at[i, 'upperB'] = row['end'] + window
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(row['start'] - window, row['start'] - ((row['start'] - dfG.iat[i-1, 4]) / 2)) # end of the gene below position [i,4]
                dfG.at[i, 'upperB'] = min(row['end'] + window, ((dfG.iat[i+1, 3] - row['end']) / 2) + row['end']) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
        outList.append(dfG)

    return(outList)

In [64]:
# # Given window size and list of ordered binding sites by chr num and size in each DF in list. Outputs list of all gene entrezid closest to esch site or 'no result'.
# def closestGeneNSitesBodyCUTOFF(DFL_BS, window):
#     groupedGenes = addWindow(window)
#     outputGeneL = []
#     pos = 0 # position in the DF we are looking at 
    
#     for sitesDF in DFL_BS: # DFL_BS is dataframe of chomosomes with N_BS ordered by the chromosome number  
#         df = groupedGenes[pos]   #.iloc[cutoff:endCut]  #endCut is the length of groupedGenes[pos]
        
#         for site in zip(sitesDF['BindingSite'].values):
#             gene = df.loc[((df.lowB - site[0]) * (df.upperB - site[0])) < 0]
            
#             if gene.empty == True:
#                 outputGeneL.append('No Result')
#             else: 
#                 outputGeneL.append(gene.iat[0,9]) # entrezid=iat
        
#         pos = pos + 1
#     return outputGeneL



def closestGeneNSitesBodyCUTOFF2(DFL_BS, window):
    groupedGenes = addWindow(window)
    outputGeneL = []
    pos = 0 # position in the DF we are looking at 
    
    for sitesDF in DFL_BS: # DFL_BS is dataframe of chomosomes with N_BS ordered by the chromosome number  
        df = groupedGenes[pos]   #.iloc[cutoff:endCut]  #endCut is the length of groupedGenes[pos]
        cutoff = 0 
        endI = len(sitesDF) - 1 
        
        for site in zip(sitesDF['BindingSite'].values):
            gene = df.loc[((df.lowB - site[0]) * (df.upperB - site[0])) < 0]
            
            if gene.empty == True:
                outputGeneL.append('No Result')
            else: 
                outputGeneL.append(gene.iat[0,9]) # entrezid=iat
                cutoff = gene.index[0]
                df.iloc[cutoff:endI]
        
        pos = pos + 1
    return outputGeneL

In [72]:
s=time.time()
op = closestGeneNSitesBodyCUTOFF2(a, 5000)
time.time()-s

13.25609803199768

In [98]:
from heapq import merge 

def closestGeneNSitesBodyCUTOFF(DFL_BS, window):
    groupedGenes = addWindow(window)
    outputGeneL = []
    pos = 0 # position in the DF we are looking at 
    s = time.time()
    for sitesDF in DFL_BS: # DFL_BS is dataframe of chomosomes with N_BS ordered by the chromosome number  
        
        df = groupedGenes[pos]   #.iloc[cutoff:endCut]  #endCut is the length of groupedGenes[pos]
        x =  merge(sitesDF['BindingSite'].values, merge(df.lowB, df.lowB))
        y =  merge(sitesDF['BindingSite'].values, merge(df.lowB, df.lowB))
        z =  merge(sitesDF['BindingSite'].values, merge(df.lowB, df.lowB))
        pos += 1 
    return time.time() - s 
#         for site in zip(sitesDF['BindingSite'].values):
#             gene = df.loc[((df.lowB - site[0]) * (df.upperB - site[0])) < 0]
            
#             if gene.empty == True:
#                 outputGeneL.append('No Result')
#             else: 
#                 outputGeneL.append(gene.iat[0,9]) # entrezid=iat
        
#         pos = pos + 1
    return outputGeneL

In [77]:
a = nBindingsites(10000)

In [99]:
v = closestGeneNSitesBodyCUTOFF(a, 5000)
v

0.0013508796691894531

In [87]:
x = [1.0,3,7.0,9]

In [88]:
x

[1.0, 3, 7.0, 9]

In [100]:
isinstance(x[1], int)

True

In [101]:
nBindingsites(100)[0]

,index,chromosome,source,type,start,end,BindingSite
0,0,1,GRCh38,chromosome,1,248956422,23832668
0,0,1,GRCh38,chromosome,1,248956422,101232611
0,0,1,GRCh38,chromosome,1,248956422,127822808
0,0,1,GRCh38,chromosome,1,248956422,172593838
0,0,1,GRCh38,chromosome,1,248956422,214785598
0,0,1,GRCh38,chromosome,1,248956422,225302598
0,0,1,GRCh38,chromosome,1,248956422,237751399
